In [ ]:
import os
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import seaborn as sns
import json 
import pickle
import bilby
import get_bilby_data as gbd
from sklearn.neighbors import KernelDensity
import random

In [ ]:
# lets compare our KDEs to our sample data 


inj_H0=72    #true H0
c=299792.45

data_copy=np.array(lum_dist)  #copy over data
h0_data=c*zt/data_copy        #convert distance samples to H0 samples 

pyplot.rcParams.update({'font.size': 18})
pyplot.figure(figsize=(12,10))

pyplot.plot(H0_axis,H0_GW_posterior, alpha=1,color=(1,0,0))  #plot KDE
pyplot.hist(h0_data, bins=200, alpha=0.4, density=True);     #plot h0_data
ax = pyplot.gca()
ylim = ax.get_ylim()
pyplot.plot([inj_H0, inj_H0], ylim, 'r--')
pyplot.ylim(ylim)
pyplot.show()

# Sampling distributions

In [ ]:
import numpy as np
import pylab as pl
import scipy
from scipy.stats import gaussian_kde
import emcee

In [ ]:
#Using Ghoshs sampling code
class H0Likelihoods:
    def __init__(self, distance_samples, z, H0, sigma):
        self.z = z
        self.H0 = H0
        self.sigma = sigma
        self.c = 2.997e8
        
        self.posterior_kde = gaussian_kde(z*c/distance_samples)
        self.prior = scipy.stats.norm(loc=self.H0, scale=self.sigma)


    def prior_val(self, d):
        return  self.prior.pdf(d)


    def likelihood_func(self, H0):
        """
            This function takes the value of H0 and the redshift, computes
        the corresponding luminosity distance then used the KDE of the
        posterior and the priors to figure out the likelihood value at
        that point
        """
        
        posterior = self.posterior_kde.evaluate(H0)
        prior = self.prior_val(H0)
        likelihood = posterior/prior
        
        return likelihood


In [ ]:
distance_data=np.array(lum_dist)
H0 = np.linspace(1, 1000, 10000)
H0_LObj = H0Likelihoods(distance_data, zt, H0_true, 5)
LIKLYHD = H0_LObj .likelihood_func(H0)

In [ ]:
pyplot.rcParams.update({'font.size': 18})
pyplot.figure(figsize=(10,8))
pyplot.plot(H0, H0_LObj.posterior_kde.evaluate(H0), label="Event: 1")
ax = pl.gca()
ylim = ax.get_ylim()
pyplot.plot([H0_true, H0_true], ylim, 'r', linewidth=4)

In [ ]:
from scipy.interpolate import interp1d

f = interp1d(H0, LIKLYHD)

In [ ]:
pl.rcParams.update({'font.size': 18})
pl.figure(figsize=(10,8))
pl.plot(H0, LIKLYHD, label="Joint - scaled down")
x = np.linspace(1, 1000, 10000)
pl.plot(x, f(x), 'r--')
pl.legend()

In [ ]:
def log_likelihood(H0):
    
    #test if H0 out of bounds
    if (H0 <= 0) or (H0 > 1000):
        return -np.infty
    
    #calculate likelihood
    L = f(H0)
    
    #test if likelihood out of log boundaries 
    if L <= 0:
        return -np.infty
    
    # else return log of likelihood
    logL = np.log(L)
    return logL  


def log_prior(H0):
    if (H0 <= 0) or (H0 > 1000):
        return -np.infty
    return 0     #why? <-------------------------- i think just for testing


def log_posterior(H0):
    lp = log_prior(H0)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(H0)

In [ ]:
ndim=1
nwalkers = 2

np.random.seed(42)
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior)

p0 = np.random.rand(nwalkers, ndim)
state = sampler.run_mcmc(p0, 100)

sampler.reset()
x=sampler.run_mcmc(state, 1000000);

In [ ]:
t = sampler.get_autocorr_time()
print(t)

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)